<a href="https://colab.research.google.com/github/BrahianCarrera/KaggleCompetition/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
# Carga el archivo kaggle.json
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai4eng-20241
!unzip udea-ai4eng-20241.zip

Saving kaggle.json to kaggle.json
 24% 5.00M/21.2M [00:00<00:00, 34.8MB/s]
100% 21.2M/21.2M [00:00<00:00, 104MB/s] 
Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [55]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import re

In [56]:
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')
df.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


# Analizar datos


In [43]:
category_cols = [i for i in df.columns if not i in df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))


ESTU_PRGM_ACADEMICO ['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES'
 'ACTIVIDAD FISICA Y DEPORTE' 'ACUICULTURA' 'ADMINISTRACION'
 'ADMINISTRACION  FINANCIERA' 'ADMINISTRACION & SERVICIO'
 'ADMINISTRACION AERONAUTICA' 'ADMINISTRACION AGROPECUARIA'
 'ADMINISTRACION AMBIENTAL'
 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES'
 'ADMINISTRACION BANCARIA Y FINANCIERA' 'ADMINISTRACION COMERCIAL'
 'ADMINISTRACION COMERCIAL Y DE MERCADEO'
 'ADMINISTRACION DE COMERCIO EXTERIOR' 'ADMINISTRACION DE EMPRESAS'
 'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL'
 'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES'
 'ADMINISTRACION DE EMPRESAS AGROPECUARIAS'
 'ADMINISTRACION DE EMPRESAS COMERCIALES'
 'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES'
 'ADMINISTRACION DE EMPRESAS TURISTICA'
 'ADMINISTRACION DE EMPRESAS TURISTICAS'
 'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS Y FINANZAS'
 'ADMINISTRACION DE GESTI

KeyboardInterrupt: 

# Eliminar Columnas

Se observo que la columna Estudio Privado de Libertad no soloe esta mal factorizada sino que todos los resultados son el mismo: No. Por tanto podemos obviar esta informacion y borrarla directamente del Dataframe.

Despues de este paso usaremos el df resultante como la base de las demas copias

In [57]:
df.isnull().sum()

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PRIVADO_LIBERTAD                 0
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_TIENEINTERNET.1              26629
FAMI_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                    0
dtype: int64

In [58]:
df = df.drop(labels =['ESTU_PRIVADO_LIBERTAD','FAMI_TIENEINTERNET.1'],axis= 1)        #Normal, Tratando de usar la mayor cantidad de informacion posible

# Renombrar las columnas

In [59]:
print(df.columns)
df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


# Convertir a binario


In [60]:
def convertir_num(valor):
    if valor == 'Si':
        return 1
    elif valor == 'No':
        return 0
    else:
        return valor  # Si no es 'Si' ni 'No', dejar el valor original

# Aplicar la función a todas las columnas del DataFrame
df = df.applymap(convertir_num)


# Limpiar datos faltantes


In [61]:
df['ESTRATO'] = df['ESTRATO'].str.replace('Estrato ', '')

In [62]:
columnas_binarias = ['TIENE_INTERNET', 'TIENE_LAVADORA', 'TIENE_AUTO',
                      'PAGA_MATRICULA', 'TIENE_COMPUTADOR']

for columna in columnas_binarias:
    moda = df[columna].mode()[0]
    df[columna].fillna(moda, inplace=True)

In [63]:
ModaEstrato = df['ESTRATO'].mode()[0]  # Obtener la moda
df['ESTRATO'].fillna(ModaEstrato, inplace=True)

ModaPadre = df['EDUCACION_PADRE'].mode()[0]
df['EDUCACION_PADRE'].fillna(ModaPadre, inplace=True)

ModaMadre = df['EDUCACION_MADRE'].mode()[0]
df['EDUCACION_MADRE'].fillna(ModaMadre, inplace=True)

ValorMatricula = df['VALOR_MATRICULA'].mode()[0]
df['VALOR_MATRICULA'].fillna(ValorMatricula, inplace=True)

HorasTrabaja = df['HORAS_TRABAJA'].mode()[0]
df['HORAS_TRABAJA'].fillna(HorasTrabaja, inplace=True)


# Convertir Datos Categoricos

Convertir las columnas de tipo objeto que ya identificamos como variables categoricas al tipo categorico




In [64]:
df["RENDIMIENTO_GLOBAL"].value_counts()

valores_rendimiento = {'alto': 4,'medio-alto': 3,'medio-bajo':2, 'bajo': 1}
df= df.replace(valores_rendimiento)



In [65]:
Columnas_categoricas = ["VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO",]

df[Columnas_categoricas] = df[Columnas_categoricas].astype("category")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   ID                  692500 non-null  int64   
 1   PERIODO             692500 non-null  int64   
 2   PROGRAMA            692500 non-null  category
 3   DEPARTAMENTO        692500 non-null  category
 4   VALOR_MATRICULA     692500 non-null  category
 5   HORAS_TRABAJA       692500 non-null  object  
 6   ESTRATO             692500 non-null  object  
 7   TIENE_INTERNET      692500 non-null  float64 
 8   EDUCACION_PADRE     692500 non-null  object  
 9   TIENE_LAVADORA      692500 non-null  float64 
 10  TIENE_AUTO          692500 non-null  float64 
 11  PAGA_MATRICULA      692500 non-null  float64 
 12  TIENE_COMPUTADOR    692500 non-null  float64 
 13  EDUCACION_MADRE     692500 non-null  object  
 14  RENDIMIENTO_GLOBAL  692500 non-null  int64   
dtypes: category(3), f

# Normalizar rangos


# **Convertir a One Hot las columnas pertinentes**
Convertir el rango de string a tipo numerico


In [72]:
print(df.isnull().sum())

ID                                                          0
PERIODO                                                     0
PROGRAMA                                                    0
DEPARTAMENTO                                                0
VALOR_MATRICULA                                             0
HORAS_TRABAJA                                               0
ESTRATO                                                     0
TIENE_INTERNET                                              0
EDUCACION_PADRE                                             0
TIENE_LAVADORA                                              0
TIENE_AUTO                                                  0
PAGA_MATRICULA                                              0
TIENE_COMPUTADOR                                            0
EDUCACION_MADRE                                             0
RENDIMIENTO_GLOBAL                                          0
VALOR_MATRICULA_Entre 1 millón y menos de 2.5 millones      0
VALOR_MA

In [68]:
codificador = OneHotEncoder()

Columnas_OneHot=['VALOR_MATRICULA']
codificacion = codificador.fit_transform(df[Columnas_OneHot])

df_codificado = pd.DataFrame(codificacion.toarray(), columns=codificador.get_feature_names_out(Columnas_OneHot))

df = pd.concat([df,df_codificado], axis = 1 )

df = df.drop(labels =['VALOR_MATRICULA'],axis= 1)


In [69]:
codificador = OneHotEncoder()

Columnas_OneHot=['HORAS_TRABAJA']
codificacion = codificador.fit_transform(df[Columnas_OneHot])

df_codificado = pd.DataFrame(codificacion.toarray(), columns=codificador.get_feature_names_out(Columnas_OneHot))

df = pd.concat([df,df_codificado], axis = 1 )

df = df.drop(labels =['HORAS_TRABAJA'],axis= 1)


In [39]:
df2 = df.copy()                                               #Dropna(), borrando todo lo que no tenga Na
df3 = df.copy()                                               #fillna() rellenando con valores lo que sea Na y se pueda rellenar
df = df.sort_index(axis=0)

In [71]:
df


,ID,PERIODO,PROGRAMA,DEPARTAMENTO,VALOR_MATRICULA,HORAS_TRABAJA,ESTRATO,TIENE_INTERNET,EDUCACION_PADRE,TIENE_LAVADORA,...,VALOR_MATRICULA_Entre 5.5 millones y menos de 7 millones,VALOR_MATRICULA_Entre 500 mil y menos de 1 millón,VALOR_MATRICULA_Menos de 500 mil,VALOR_MATRICULA_Más de 7 millones,VALOR_MATRICULA_No pagó matrícula,HORAS_TRABAJA_0,HORAS_TRABAJA_Entre 11 y 20 horas,HORAS_TRABAJA_Entre 21 y 30 horas,HORAS_TRABAJA_Menos de 10 horas,HORAS_TRABAJA_Más de 30 horas
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,3,1.0,Técnica o tecnológica incompleta,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,3,0.0,Técnica o tecnológica completa,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,3,1.0,Secundaria (Bachillerato) completa,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,4,1.0,No sabe,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,3,1.0,Primaria completa,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,2,1.0,Secundaria (Bachillerato) completa,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,3,1.0,Primaria incompleta,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,3,1.0,Secundaria (Bachillerato) completa,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,1,0.0,Primaria completa,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
